# Stage 1: Object Detection ------------------------------------------------------------------------

In [4]:
import sys
import tensorflow as tf
import numpy as np
from support_files.object_detection import ObjectDetection
import cv2
import os
from math import sqrt, acos, pi
from PIL import Image

# ------------------------------------------------------------------------------------------------------------------------
MODEL_FILENAME = 'support_files/models/Object_Detection_Model.pb'
LABELS_FILENAME = 'support_files/models/Object_Detection_Labels.txt'

# ------------------------------------------------------------------------------------------------------------------------
class Point():
    x = None; y = None

    def __init__(self, x, y):
        self.x, self.y = x, y

    def __str__(self):
        return "%6.1f, %6.1f" % (self.x, self.y)

    def __eq__(self, obj):
        return obj.x == self.x and obj.y == self.y

    def distance_to_point(self, p):
        return sqrt((self.x-p.x)**2+(self.y-p.y)**2)

    def faces_line(self, line):
        return point_faces_edge(line, self)
    
# ------------------------------------------------------------------------------------------------------------------------
class Rect():

    # Screen coordinates
    l_top  = None; r_top  = None; l_bot  = None; r_bot  = None
    center = None; width  = None; height = None

    def __init__(self, x, y, width, height):
        assert width>0
        assert height>0
        self.l_top  = Point(x, y)
        self.r_top  = Point(x+width, y)
        self.r_bot  = Point(x+width, y+height)
        self.l_bot  = Point(x, y+height)
        self.center = Point(x+width/float(2), y+height/float(2))
        self.width  = width
        self.height = height

    def __str__(self):
        str=("(%4d,%4d)              (%4d,%4d)\n"
             "      .-----------------------.\n"
             "      |                       |\n"
             "      |                %6.1f |\n"
             "      |       %6.1f          |\n"
             "      '-----------------------'\n"
             "(%4d,%4d)              (%4d,%4d)"
             )
        nums=(self.l_top.x, self.l_top.y, self.r_top.x, self.r_top.y, self.height, self.width, self.l_bot.x, self.l_bot.y, self.r_bot.x, self.l_bot.y )
        return str % nums

    def __iter__(self):
        yield self.l_top
        yield self.r_top
        yield self.r_bot
        yield self.l_bot

    def iter_edges(self):
        yield self.l_top, self.r_top
        yield self.r_top, self.r_bot
        yield self.r_bot, self.l_bot
        yield self.l_bot, self.l_top

    def is_point_inside_rect(self, point):
        return (self.l_top.x <= point.x <= self.r_top.x and self.l_top.y <= point.y <= self.l_bot.y)

    def overlaps_with(self, rect):
        for corner in rect:
            if self.is_point_inside_rect(corner):
                return True
        for corner in self:
            if rect.is_point_inside_rect(corner):
                return True
            return False
        
# ------------------------------------------------------------------------------------------------------------------------
path = 'support_files/receipts/' # This is the folder where all the receipts are stored
files = []
image_names = []
# r=root, d=directories, f = files
for r, d, f in os.walk(path):
    for file in f:
        if '.jpg' in file:
            files.append(os.path.join(r, file))
            image_names.append(os.path.join(file))
            
for i in range (0, len(image_names)):
    fname = image_names[i]
    fname = fname[0:len(fname) - 4]
    
    if os.path.isdir('support_files/cropped_images/' + fname) == False:
        os.makedirs('support_files/cropped_images/' + fname)
        
# ------------------------------------------------------------------------------------------------------------------------
def four_point_transform(image, tl, tr, bl, br):
    rect = np.array(np.zeros((4, 2), dtype="float32"))
    rect[0][0] = tl[1]; rect[0][1] = tl[0]
    rect[1][0] = tr[1]; rect[1][1] = tr[0]
    rect[2][0] = br[1]; rect[2][1] = br[0]
    rect[3][0] = bl[1]; rect[3][1] = bl[0]

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype="float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

# ------------------------------------------------------------------------------------------------------------------------
def unzipDictionary(Dict):

    probability = Dict['probability']
    tagId = Dict['tagId']
    tagName = Dict['tagName']
    boundingBox = Dict['boundingBox']
    left = boundingBox['left']
    top = boundingBox['top']
    width = boundingBox['width']
    height = boundingBox['height']

    return probability, tagId, tagName, left, top, width, height

# ------------------------------------------------------------------------------------------------------------------------
def cropAnnotations(image, filename, predictions):
    (r, c, l) = np.shape(image)

    if len(predictions) > 0:
        j = 0;  k = 0
        Dict = ['None'] * len(predictions)
        tempPredictions = ['None'] * len(predictions)

        for i in range(0, len(predictions)):
            box_parameters = predictions[i]
            if box_parameters['tagId'] == 3:
                Dict[j] = box_parameters
                j += 1
            else:
                tempPredictions[k] = box_parameters
                k += 1

        Dict = Dict[0:j]
        tempPredictions = tempPredictions[0:k]

        if len(Dict) > 1:
            tempDict = Dict.copy()
            n = len(Dict)
            for x in range(0, int(n)):
                for i in range(0, len(tempDict)):
                    if len(tempDict) <= i:
                        break

                    box_parameters = tempDict[i]
                    c1 = box_parameters

                    _, _, _, left, top, width, height = unzipDictionary(box_parameters)
                    r1 = Rect(int(left*c), int(top*r), int(width*c), int(height*r))
                    
#                     print("Rect : ", (int(left*c), int(top*r), int(width*c), int(height*r)))
#                     print(r1)
                    tempDict.pop(i)

                    if len(tempDict) > 0:
                        for j in range (0, len(tempDict)):
                            box_parameters = tempDict[j]
                            _, _, _, left, top, width, height = unzipDictionary(box_parameters)
                            r2 = Rect(int(left*c), int(top*r), int(width*c), int(height*r))

                            if r1.overlaps_with(r2) == True:
                                if c1['probability'] < box_parameters['probability']:
                                    Dict.pop(Dict.index(c1))
                                else:
                                    Dict.pop(Dict.index(box_parameters))
                                break
                    tempDict = Dict.copy()
            predictions = tempPredictions + Dict

        #-----------------------------------------------------------------------
        j = 0
        filename = filename[0:len(filename)-4]
        for i in range(0, len(predictions)):
            box_parameters = predictions[i]
            _, tagId, tagName, left, top, width, height = unzipDictionary(box_parameters)

            box = [int(left*c), int(top*r), int(int(left*c) + width*c), int(int(top*r) + height*r)]
            tl = (box[1], box[0]); br = (box[3], box[2]); tr = (box[1], box[2]); bl = (box[3], box[0])
            img0 = four_point_transform(image, tl, tr, bl, br)

            if tagId == 0:
                if os.path.isdir("support_files/cropped_images/" + filename + "/mID/") == False:
                    os.makedirs("support_files/cropped_images/" + filename + "/mID/")
                cv2.imwrite("support_files/cropped_images/" + filename + "/mID/" + filename + "_" + str(tagName) + ".jpg", img0)

            elif tagId == 1:
                if os.path.isdir("support_files/cropped_images/" + filename + "/receiverID/") == False:
                    os.makedirs("support_files/cropped_images/" + filename + "/receiverID/")
                cv2.imwrite("support_files/cropped_images/" + filename + "/receiverID/" + filename + "_" + str(tagName) + "ID" + ".jpg", img0)

            elif tagId == 2:
                if os.path.isdir("support_files/cropped_images/" + filename + "/senderID/") == False:
                    os.makedirs("support_files/cropped_images/" + filename + "/senderID/")
                cv2.imwrite("support_files/cropped_images/" + filename + "/senderID/" + filename + "_" + str(tagName) + "ID" + ".jpg", img0)

            elif tagId == 3:
                tagName = str(tagName) + "_" + str(j)
                if os.path.isdir("support_files/cropped_images/" + filename + "/tel/") == False:
                    os.makedirs("support_files/cropped_images/" + filename + "/tel/")
                cv2.imwrite("support_files/cropped_images/" + filename + "/tel/" + filename + "_" + str(tagName) + ".jpg", img0)
                j += 1
                
# ------------------------------------------------------------------------------------------------------------------------
class TFObjectDetection(ObjectDetection):
    """Object Detection class for TensorFlow
    """

    def __init__(self, graph_def, labels):
        super(TFObjectDetection, self).__init__(labels)
        self.graph = tf.Graph()
        with self.graph.as_default():
            input_data = tf.compat.v1.placeholder(tf.float32, [1, None, None, 3], name='Placeholder')
            tf.import_graph_def(graph_def, input_map={"Placeholder:0": input_data}, name="")

    def predict(self, preprocessed_image):
        inputs = np.array(preprocessed_image, dtype=np.float)[:, :, (2, 1, 0)]  # RGB -> BGR

        with tf.compat.v1.Session(graph=self.graph) as sess:
            output_tensor = sess.graph.get_tensor_by_name('model_outputs:0')
            outputs = sess.run(output_tensor, {'Placeholder:0': inputs[np.newaxis, ...]})
            return outputs[0]
        
# ------------------------------------------------------------------------------------------------------------------------
def main_algo():
        
    # Load a TensorFlow model
    graph_def = tf.compat.v1.GraphDef()
    with open(MODEL_FILENAME, 'rb') as f:
        graph_def.ParseFromString(f.read())
        
    # Load labels
    with open(LABELS_FILENAME, 'r') as f:
        labels = [l.strip() for l in f.readlines()]

    od_model = TFObjectDetection(graph_def, labels)

    # This For Loop is added by Niraj ------------------------------------------
    for image_filename in image_names:
        print('------------------------------------------- ' + image_filename + ' -------------------------------------------')

        image = Image.open(path + image_filename)
        predictions = od_model.predict_image(image)

        img = cv2.imread(path + image_filename)
        cropAnnotations(img, image_filename, predictions)
        
# ------------------------------------------------------------------------------------------------------------------------
main_algo()
print(" First Processing Complete.")

------------------------------------------- 1 (1).jpg -------------------------------------------
------------------------------------------- 1 (2).jpg -------------------------------------------
------------------------------------------- 1 (3).jpg -------------------------------------------
------------------------------------------- 1 (4).jpg -------------------------------------------
------------------------------------------- 1 (5).jpg -------------------------------------------
------------------------------------------- 1 (6).jpg -------------------------------------------
------------------------------------------- 2 (1).jpg -------------------------------------------
------------------------------------------- 2 (2).jpg -------------------------------------------
------------------------------------------- 2 (3).jpg -------------------------------------------
------------------------------------------- 2 (4).jpg -------------------------------------------
--------------------

# Stage 2: 1D image crop ----------------------------------------------------------------------------

# Load names of the files

In [5]:
def order_points(pts):
    rect = np.zeros((4, 2), dtype = "float32")
    s = pts.sum(axis = 1)
    rect[0] = pts[np.argmin(s)]
    rect[2] = pts[np.argmax(s)]
    diff = np.diff(pts, axis = 1)
    rect[1] = pts[np.argmin(diff)]
    rect[3] = pts[np.argmax(diff)]
    return rect

def four_point_transform0(image, pts):
    a = pts[0]
    b = pts[1]
    pts = [a, b, (a[0], b[1]), (b[0], a[1])]

    pts = np.array(eval(str(pts)), dtype = "float32")
    rect = order_points(pts)
    (tl, tr, br, bl) = rect

    widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
    widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
    maxWidth = max(int(widthA), int(widthB))

    heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
    heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
    maxHeight = max(int(heightA), int(heightB))

    dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype = "float32")

    M = cv2.getPerspectiveTransform(rect, dst)
    warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))
    return warped

# ------------------------------------------------------------------------------------------------------------------------
def overlapping_area(rectangle1, rectangle2):
    r1 = Rect(rectangle1[0], rectangle1[1], rectangle1[2]-rectangle1[0], rectangle1[3]-rectangle1[1])
    r2 = Rect(rectangle2[0], rectangle2[1], rectangle2[2]-rectangle2[0], rectangle2[3]-rectangle2[1])

    if rectangle1 == rectangle2 :
        return (rectangle1[2]-rectangle1[0]) * (rectangle1[3]-rectangle1[1])

    if (r1.overlaps_with(r2)) == True:

        a1 = (rectangle1[2]-rectangle1[0]) * (rectangle1[3]-rectangle1[1])
        a2 = (rectangle2[2]-rectangle2[0]) * (rectangle2[3]-rectangle2[1])


        small_rect = 0
        if a1 <= a2:
            small_rect = rectangle1
            big_rect   = r2
        else:
            small_rect = rectangle2
            big_rect   = r1

        count_point = 0
        for x in range (small_rect[0], small_rect[2]):
            for y in range (small_rect[1], small_rect[3]):
                #print((x, y))
                if big_rect.is_point_inside_rect(Point(x, y)) == True:
                    count_point += 1
        return count_point

    elif (r1.overlaps_with(r2)) == False:
        return -1

def area_of_rectangle(rectangle1):
    return (rectangle1[2] - rectangle1[0]) * (rectangle1[3] - rectangle1[1])

def clean_the_list(list1):

    max_range = len(list1)
    for rty in range (0, 30):
        for rect10 in range (1, len(list1)-1):

            if area_of_rectangle(list1[rect10]) >= area_of_rectangle(list1[rect10+1]):
                bigger_rectangle = list1[rect10]
                smaller_rectangle = list1[rect10+1]
            else:
                bigger_rectangle = list1[rect10+1]
                smaller_rectangle = list1[rect10]


            if overlapping_area(list1[rect10], list1[rect10+1]) >= 0.3 * area_of_rectangle(bigger_rectangle):
                list1.remove(smaller_rectangle)
                break

    return list1
# ------------------------------------------------------------------------------------------------------------------------

path = 'support_files/cropped_images/'
fields_filename_all = [0] * len(image_names)
image_names0 = image_names.copy()
for i in range (0, len(image_names0)):
    a = image_names0[i]
    fields_filename_all[i] = path + a[0:len(a)-4] + '/'

# ------------------------------------------------------------------------------------------------------------------------

image_paths = []
only_image_name = []
for fn in (fields_filename_all):
    path0 = fn + 'tel/'
    file_names = []  
    
    if os.path.isdir(path0) == True:
        arr = os.listdir(path0)
        
        if len(arr) != 0:
            new_list = []
            for i in range (0, len(arr)):
                a = arr[i]
                if len(a) > 4 and a[len(a)-4:len(a)] == '.jpg':
                    new_list.append(a)
            
            if len(new_list) != 0:
                
                for i in range (0, len(new_list)):        
                    a = new_list[i]
                    new_path = fn + 'tel/'
                    new_path1 = fn + 'tel/' + a[0:len(a)-4] + '/'
                    
                    if os.path.isdir(new_path1) == False:
                        os.makedirs(new_path1)
                        
                    if a != []:
                        image_paths.append(new_path+a)
                        only_image_name.append(a)

In [6]:
# %load Object_detection_image.py

# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import pickle

print("Second Processing Starts...")

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")

# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Name of the directory containing the object detection module we're using
MODEL_NAME = 'inference_graph'
IMAGE_NAME = 'image_test/image6.jpg'
image_file_name = "image6.jpg"

# Grab path to current working directory
CWD_PATH = os.getcwd()

# Path to frozen detection graph .pb file, which contains the model that is used
# for object detection.
PATH_TO_CKPT = os.path.join(CWD_PATH,MODEL_NAME,'frozen_inference_graph.pb')

# Path to label map file
PATH_TO_LABELS = os.path.join(CWD_PATH,'training','labelmap.pbtxt')

# Path to image
PATH_TO_IMAGE = os.path.join(CWD_PATH,IMAGE_NAME)

# Number of classes the object detector can identify
NUM_CLASSES = 10

# Load the label map.
# Label maps map indices to category names, so that when our convolution
# network predicts `5`, we know that this corresponds to `king`.
# Here we use internal utility functions, but anything that returns a
# dictionary mapping integers to appropriate string labels would be fine
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Define input and output tensors (i.e. data) for the object detection classifier

# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

# Load image using OpenCV and
# expand image dimensions to have shape: [1, None, None, 3]
# i.e. a single-column array, where each item in the column has the pixel RGB value

# Niraj's Code -----------------------------------------------------------------------------

for i in range (0, len(image_paths)):
    print(str(i+1) + "/" + str(len(image_paths)))
    image = cv2.imread(image_paths[i], 1)
    image = cv2.resize(image, (265, 100), interpolation = cv2.INTER_AREA)
    
    
    
    image0 = image.copy()
    
    if len(np.shape(image)) == 3:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    _, image = cv2.threshold(image, cv2.mean(image)[0]-25, 255, cv2.THRESH_BINARY_INV)
    image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    
    image_expanded = np.expand_dims(image, axis=0)

    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_expanded})

    # Draw the results of the detection (aka 'visulaize the results')

    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=1,
        min_score_thresh=0.60,
        filename=image_file_name)

    
    with open("file1.txt", "rb") as fp:
        list0 = pickle.load(fp)

    # rearrage rectangle coordinates
    arr0 = []
    for j in range (1, len(list0)):
        arr0.append((list0[j])[0])
    arr0.sort()
    new_list0 = [list0[0]]
    for m in range (0, len(arr0)):
        for j in range(1, len(list0)):
            if arr0[m] == (list0[j])[0]:
                new_list0.append(list0[j])
    
    new_list0 = clean_the_list(new_list0)
    
    
    specific_path = image_paths[i]
    specific_path = specific_path[0:len(specific_path)-4]
    name00 = only_image_name[i]
    nn = 0
    for j in range (1, len(new_list0)):
        coord = new_list0[j]
        
        cropped_small_image = four_point_transform0(image0.copy(), [(coord[0], coord[1]), (coord[2], coord[3])])
        cv2.imwrite(specific_path + "/" + name00[0:len(name00)-4] + "_" 
                    + str(nn) + '.jpg', cropped_small_image)

        cv2.imwrite("C:/Users/mapl3381/Desktop/demo/1/object_detection/support_files/good/" + name00[0:len(name00)-4] + "_" 
                    + str(nn) + '.jpg', cropped_small_image)
        nn += 1

print("Second Processing Complete.")

Second Processing Starts...
1/40
2/40
3/40
4/40
5/40
6/40
7/40
8/40
9/40
10/40
11/40
12/40
13/40
14/40
15/40
16/40
17/40
18/40
19/40
20/40
21/40
22/40
23/40
24/40
25/40
26/40
27/40
28/40
29/40
30/40
31/40
32/40
33/40
34/40
35/40
36/40
37/40
38/40
39/40
40/40
Second Processing Complete.


# Step 3: Good/Bad Claasification and Digit Recognition ------------------------------

In [8]:
from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import pickle

import os
import numpy as np
import cv2
if os.path.isdir('support_files/receipts/') == True:
    image_names= os.listdir('support_files/receipts/')

for i in range (0, 4):
    print(image_names[i])
    
# ---------------------------------------------------------------------------------------------------------------

with open("support_files/models/Final_Good_Bad.pkl", 'rb') as file:  
    model_goodBad = pickle.load(file)
    
with open("support_files/models/Final_Digit_Prediction.pkl", 'rb') as file:  
    model_digit_prediction = pickle.load(file) 
# ------

def x_predict_return(x_predict, img_rows, img_cols):
    if K.image_data_format() == 'channels_first':
        x_predict = x_predict.reshape(x_predict.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        x_predict = x_predict.reshape(x_predict.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1)

    x_predict = x_predict.astype('float32')
    x_predict /= 255
    return x_predict

1 (1).jpg
1 (2).jpg
1 (3).jpg
1 (4).jpg


In [9]:
img_rows, img_cols = 28, 28
path = 'support_files/cropped_images/'

for i in range (0, len(image_names)):
    b = image_names[i]
    b = b[0:len(b)-4]
    path0 = path + b + '/tel/'
    
    #print("------------------------- " + path + b + " -------------------------")
    print(" --------------------------------------------------------------------- ")
    if os.path.isdir(path0) == True:
        arr = os.listdir(path0)
        
        if len(arr) != 0:    # proceed if 'support_files/cropped_images/Image_name/tel/' is not empty
            new_list = []
            tel = 0
            for j in range (0, len(arr)):
                a = arr[j]
                if len(a) > 4 and a[len(a)-4:len(a)] != '.jpg':
                    path1 = path0 + a + '/'
                    image_path_display =  path0 + a + '.jpg'
                    
                    if os.path.isdir(path1) == True:
                        arr1 = os.listdir(path1)

                        count = 0
                        for k in range (0, len(arr1)):
                            c = arr1[k]
                            if c[len(c)-4:len(c)] == '.jpg':
                                count += 1

                        if count != 0:    # proceed if 'support_files/cropped_images/Image_name/tel/Image_name_subname' is not empty
                            list000 = []
                            mat0 = np.zeros((count, img_rows, img_cols), dtype=int)

                            mm = 0
                            for k in range (0, len(arr1)):                
                                c = arr1[k]
                                if c[len(c)-4:len(c)] == '.jpg':                    
                                    img0 = cv2.imread(path1 + arr1[k])
                                    if len(np.shape(img0)) == 3:
                                        img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
                                    img0 = cv2.resize(img0, (img_cols, img_rows), interpolation = cv2.INTER_AREA)
                                    mat0[mm, :, :] = img0
                                    list000.append(arr1[k])
                                    mm += 1
                            # ---------------------------------------------------------------------------------------
                            x_predict = mat0.copy()
                            x_predict = x_predict_return(x_predict, img_rows, img_cols)
                            # ---------------------------------------------------------------------------------------
                            prediction = model_goodBad.predict_classes(x_predict, verbose=0)

                            if os.path.isdir(path1 + 'good/') == False:
                                os.makedirs(path1 + 'good/')
                            if os.path.isdir(path1 + 'bad/') == False:
                                os.makedirs(path1 + 'bad/')

                            length_prediction_images = []
                            lll = 0
                            for k in range (0, len(prediction)):
                                img0 = cv2.imread(path1 + list000[k])
                                if prediction[k] == 1:
                                    cv2.imwrite(path1 + 'good/' + list000[k], img0)
                                    
                                elif prediction[k] == 0:                    
                                    cv2.imwrite(path1 + 'bad/' + list000[k], img0) 
                            
                            # digit prediction =================================================================
                            # -----------------------------------------------------------------------------------
                            if os.path.isdir(path1 + 'good/') == True:
                                arr2 = os.listdir(path1 + 'good/')
                                
                                if len(arr2) != 0:
                                    mat0 = np.zeros((len(arr2), img_rows, img_cols), dtype=int)
                                    for q in range (0, len(arr2)):                                        
                                        img0 = cv2.imread(path1 + 'good/' + arr2[q])                                        
                                        if len(np.shape(img0)) == 3:
                                            img0 = cv2.cvtColor(img0, cv2.COLOR_BGR2GRAY)
                                        _, img0 = cv2.threshold(img0, int(cv2.mean(img0)[0]-15), 255, cv2.THRESH_BINARY_INV)
                                        img0 = np.pad(img0, [(4, ), (2, )], mode='constant')
                                        img0 = cv2.resize(img0, (img_cols, img_rows), interpolation = cv2.INTER_AREA)
                                        
#                                         cv2.imshow('image', img0)
#                                         cv2.waitKey(0)
#                                         cv2.destroyAllWindows()
                                        
                                        
                                        mat0[q][:][:] = img0
                                
                                x_predict = mat0.copy()
                                
                                if len(arr2) != 0:
                                    x_predict = x_predict_return(x_predict, img_rows, img_cols)
                            
                                    digit_prediction = model_digit_prediction.predict_classes(x_predict, verbose=0)
                                    print(digit_prediction)
                                   
                                    image_show = cv2.imread(image_path_display)
                                    cv2.imshow('image', image_show)
                                    cv2.waitKey(0)
                                    cv2.destroyAllWindows()

 --------------------------------------------------------------------- 
[9 2 8 7 1 0 7 9 8 8 3]
[0 9 1 7 9 9 8 3 4 1]
 --------------------------------------------------------------------- 
[0 2 3 4 5 0 3 1]
[2 7 4 6 4 1 2 8]
 --------------------------------------------------------------------- 
[7 7 3 0 9 0 9 1]
[2 7 8 9 2 7 7 1]
 --------------------------------------------------------------------- 
[8 7 7 5 1 3 2]
[5 2 5 3 8 2 8 6 8]
 --------------------------------------------------------------------- 
[8 7 7 5 5 7 5]
[2 7 2 3 5 3 1]
 --------------------------------------------------------------------- 
[8 7 7 2 5 5 7 5]
[0 9 2 9 7 5 2 4 7]
 --------------------------------------------------------------------- 
[2 7 3 1 1 5 7 5]
[9 0 3 9 7 6 7 5 5 4 3]
 --------------------------------------------------------------------- 
[0 5 4 2 2 7 8 1 1]
[2 7 1 3 7 7 5 3]
 --------------------------------------------------------------------- 
[2 5 8 1 8 3 0 3]
[8 1 0 3 7 8 9 7]
 -----------

In [11]:
import os
with open("file1.txt", "rb") as fp:
    
    list0 = pickle.load(fp)
    print(list0)

['image6.jpg', (91, 18, 115, 72), (154, 22, 178, 75), (69, 19, 93, 70), (133, 26, 158, 78), (198, 27, 222, 81), (46, 18, 73, 74), (175, 28, 202, 81), (112, 24, 138, 79)]
